In [1]:
import pandas as pd
import numpy as np

In [6]:
calendar = pd.read_csv('./seattle/calendar.csv')
listings = pd.read_csv('./seattle/listings.csv')
reviews = pd.read_csv('./seattle/reviews.csv')

# Calendar 

In [11]:
calendar.head()

,listing_id,date,available,price
0,241032,2016-01-04,t,$85.00
1,241032,2016-01-05,t,$85.00
2,241032,2016-01-06,f,NaN
3,241032,2016-01-07,f,NaN
4,241032,2016-01-08,f,NaN


In [30]:
calendar.dtypes

listing_id             int64
date          datetime64[ns]
available             object
price                 object
dtype: object

In [29]:
calendar['date'] = pd.to_datetime(calendar['date'], format = '%Y-%m-%d')

In [45]:
#check if all listing ids have the same number of dates

count_dates = calendar[['listing_id','date']].groupby('listing_id').count()
count_dates['date'].unique()

array([365], dtype=int64)

In [49]:
calendar.available.unique()

array([1, 0], dtype=int64)

In [52]:
# replace 'available' column data with numeric binary data
try:
    calendar['available'] = calendar['available'].replace({'t':1,'f':0})
except:
    pass
calendar.available.unique()

array([1, 0], dtype=int64)

In [54]:
calendar[['listing_id','available']].groupby('listing_id').mean()

,available
listing_id,
3335,0.846575
4291,1.000000
5682,0.846575
6606,1.000000
7369,0.145205
9419,1.000000
9460,0.156164
9531,0.493151
9534,0.717808


# Reviews

In [8]:
reviews.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,7202016,38917982,2015-07-19,28943674,Bianca,Cute and cozy place. Perfect location to every...
1,7202016,39087409,2015-07-20,32440555,Frank,Kelly has a great room in a very central locat...
2,7202016,39820030,2015-07-26,37722850,Ian,"Very spacious apartment, and in a great neighb..."
3,7202016,40813543,2015-08-02,33671805,George,Close to Seattle Center and all it has to offe...
4,7202016,41986501,2015-08-10,34959538,Ming,Kelly was a great host and very accommodating ...


In [55]:
listings.shape

(3818, 92)